In [37]:
import pandas as pd
import datetime as dt
import time
from datetime import datetime, timedelta
import os
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

In [38]:
# Задаем сегодняшнюю дату
today = datetime.now()

# Задаем годовую и дневную дельту
year = timedelta(days = 365)
day = timedelta(days = 1)

# Зададим переменную для создания пути
now = str(today.date())


### Загрузим таблицу компаний

In [39]:
company = pd.read_excel('company.xlsx') # По этой таблице будем проводить фильтрацию компаний

# Очистим названия компаний от разных кавычек и пробелов вначале и конце строки
def new_comp(x):
    x = x.replace('«','"').replace('»','"').strip()
    return x

company['company'] = company['company'].apply(new_comp)
company['industry'] = company['industry'].apply(new_comp)
company = company.drop('Unnamed: 0',axis=1)

### Загрузим таблицу регионов

In [40]:
regions = pd.read_excel('regions.xlsx', header = None)

# Очистили ee от шлака
regions[0] = regions[0].apply(lambda x: x.strip()) # в таком виде эта таблица понадобится дальше для конкатинации

# Сделаем на ее основе еще одну таблицу как справочник по регионам
region_df = regions.copy()

# задаем функцию для выявления ОЭС
def filter_region(str):
    if 'ОЭС' in str:
        return str

# Применяем
region_df['UES'] = region_df[0].apply(filter_region)

# Задаем функцию для заполнения пустых строк регионов
def fill_empty(col):
    col = col.fillna(0)
    for i in range(len(col)):
        if col.iloc[i] == 0:
            col.iloc[i] = col.iloc[i-1]
        else:
            continue
    return col

# Применяем
region_df['UES'] = fill_empty(region_df['UES'])

# Дополнительно убираем ненужные строки, сбрасываем индексы, переименовываем колонки
region_df = region_df.drop(region_df[region_df[0].str.contains('ОЭС', regex=True)==True].index)
region_df = region_df.reset_index()
region_df = region_df.drop(['index'],axis=1)
region_df.columns = ['region','UES']

# Энергосистемы

 Здесь мы загрузим, обратобаем и проанализируем данные по потреблению энергосистем, воходящих в единую энергосистему России

### UES Загрузим таблицу с оперативными данными потребления энергосистем

In [41]:
# Загружаем основной датафрэйм
df_ues1 = pd.read_excel('анализ изменения потребления.xlsx', sheet_name=0)
df_ues2 = pd.read_excel('анализ изменения потребления.xlsx', sheet_name=1)

df_ues2 = df_ues2.drop(['Unnamed: 0','Unnamed: 1'],axis=1)

df_ues = pd.concat([df_ues1,df_ues2],axis=1)

df_ues['Unnamed: 0'] = df_ues['Unnamed: 0'].apply(lambda x: x.strip())

In [42]:
# Сделаем три отдельных таблицы:

# С данными по потреблению за 20 год
df_ues20 = df_ues[df_ues['Unnamed: 0']=='Потребление ЭЭ 2020']
df_ues20.reset_index(inplace=True)
df_ues20 = df_ues20.drop('index', axis=1)

# С данными по потреблению за 19 год
df_ues19 = df_ues[df_ues['Unnamed: 0']=='Потребление ЭЭ 2019']
df_ues19.reset_index(inplace=True)
df_ues19 = df_ues19.drop('index', axis=1)

# С приведенными по температуре данными по потреблению за 20 год
df_uespr = df_ues[df_ues['Unnamed: 0']=='Приведение ЭЭ опер. к прошлому году']
df_uespr.reset_index(inplace=True)
df_uespr = df_uespr.drop('index', axis=1)

In [43]:
# Присоединили к таблицам с потреблением имена регионов
df_ues19 = pd.concat([regions,df_ues19],axis=1)
df_ues20 = pd.concat([regions,df_ues20],axis=1)
df_uespr = pd.concat([regions,df_uespr],axis=1)

# Очистили
df_ues19 = df_ues19.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
df_ues20 = df_ues20.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
df_uespr = df_uespr.drop(['Unnamed: 0','Unnamed: 1'],axis=1)

# Пишем новые индексы на 19 и 20 года
index_ues19 = pd.DatetimeIndex(pd.Series(df_ues.iloc[0][2:]))
index_ues20 = pd.DatetimeIndex(pd.Series(df_ues.iloc[1][2:]))

In [44]:
# Транспонитуем
df_ues19 = df_ues19.T
df_ues20 = df_ues20.T
df_uespr = df_uespr.T

In [45]:
# Сделаем названия компаний заголовками

new_header_ues20 = df_ues20.iloc[0]

df_ues20 = df_ues20[1:]
df_ues20.columns = new_header_ues20
# Сделаем новые индексы
df_ues20.index = index_ues20

df_ues19 = df_ues19[1:]
df_ues19.columns = new_header_ues20
# Сделаем новые индексы
df_ues19.index = index_ues19

df_uespr = df_uespr[1:]
df_uespr.columns = new_header_ues20
# Сделаем новые индексы
df_uespr.index = index_ues20

### Ведем переменную последней даты, которая есть в рассматриваемых данных

In [46]:
lastday = df_ues20.index[-1].day

### UES Заресемплим данные в понедельные

In [47]:
df_ues20_week = df_ues20.resample('W').sum()
df_ues20_week.index = df_ues20_week.index.week

df_ues19_week = df_ues19.resample('W').sum()
df_ues19_week.index = df_ues19_week.index.week

df_uespr_week = df_uespr.resample('W').sum()
df_uespr_week.index = df_uespr_week.index.week

### UES Заресемплим данные в помесячные

In [48]:
df_ues20_month = df_ues20.resample('M').sum()
df_ues20_month.index = df_ues20_month.index.month

df_ues19_month = df_ues19.resample('M').sum()
df_ues19_month.index = df_ues19_month.index.month

df_uespr_month = df_uespr.resample('M').sum()
df_uespr_month.index = df_uespr_month.index.month

### UES Разности по неделям

In [49]:
# Абсолютные значения
abs_fall_week = df_ues20_week - df_ues19_week
abs_fall_week_t = df_uespr_week - df_ues19_week

# Относительные занчаения
per_fall_week = abs_fall_week*100/df_ues19_week
per_fall_week_t = abs_fall_week_t*100/df_ues19_week

# Округляем
per_fall_week_t = per_fall_week_t.astype(float).round(1)

### UES Разности по месяцам

In [50]:
# Абсолютные значения
abs_fall_month = df_ues20_month - df_ues19_month
abs_fall_month_t = df_uespr_month - df_ues19_month

# Относительные занчаения
per_fall_month = abs_fall_month*100/df_ues19_month
per_fall_month_t = abs_fall_month_t*100/df_ues19_month

### UES Загрузим таблицу с фактическими данными потребелия по месяцам

In [51]:
mes_ues = pd.read_excel('Приведен_помес_потребл_ЕЭС_2019_2020.xlsx', sheet_name=0)

mes_ues = mes_ues.iloc[1:]

mes_ues.index = mes_ues['Потебление России по месяцамза 2019 и 2020 год, млн. кВтч']
mes_ues = mes_ues.drop('Потебление России по месяцамза 2019 и 2020 год, млн. кВтч',axis=1)

# Прописываем столбец с датами
mes_ues.iloc[1] = mes_ues.iloc[1] + mes_ues.iloc[0]

def make_datetime(x):
    if isinstance(x, str):
        x = x.replace('Январь','jan ')
        x = x.replace('Февраль','feb ')
        x = x.replace('Март','mar ')
        x = x.replace('Апрель','apr ')
        x = x.replace('Май','may ')
        x = x.replace('Июнь','jun ')
        x = x.replace('Июль','jul ')
        x = x.replace('Август','aug ')
        x = x.replace('Сентябрь','sep ')
        x = x.replace('Октябрь','oct ')
        x = x.replace('Ноябрь','nov ')
        x = x.replace('Декабрь','dec ')
        #x = datetime.strptime(x, '%b %Y')
        #x = datetime.fromtimes_uestamp(x)
    return x

mes_ues.iloc[1] = mes_ues.iloc[1].apply(make_datetime)
mes_ues.iloc[1] = mes_ues.iloc[1].apply(lambda x: datetime.strptime(x, '%b %Y'))

# Прописываем новые колонки
mes_ues.columns = pd.DatetimeIndex(mes_ues.iloc[1])

mes_ues = mes_ues.iloc[2:]
 
# Список энергосистем 
list_UES = ['ЕЭС России', 'ОЭС Центра', 'ОЭС Средней Волги', 'ОЭС Урала', 'ОЭС Северо-Запада', 'ОЭС Юга',
           'ОЭС Сибири', 'ОЭС Востока']

# ДФ с фактическими показателями потребления по всем регионам
mes_ues_all = mes_ues.T
list_all = mes_ues_all.columns

# Приведем названия субъектов к общему виду
list_all_new = []
for x in list_all:
    x = x.replace('э/с','ЭС')
    x = x.replace('Ханты-Мансийского и Ямало-Ненецкого автономных округов','ХМАО и ЯНАО')
    list_all_new.append(x)

# Перепропишем названия столбцов
mes_ues_all.columns = list_all_new

mes_ues_20_month = mes_ues_all.loc['2020-01-01':] 
mes_ues_19_month = mes_ues_all.iloc[:len(mes_ues_20_month)]

# ДФ со снижениями по ВСЕМ регионам
mes_ues_percent = pd.DataFrame(mes_ues_20_month.values / mes_ues_19_month.values*100 - 100)
mes_ues_percent.columns = list_all_new
#mes_ues_percent.index = ['январь','февраль','март','апрель','май','июнь','июль'] # 

# ДФ со снижениями по ОЭС
mes_UES = mes_ues_percent.T[mes_ues_percent.T.index.isin(list_UES)]
mes_UES = mes_UES.T[list_UES]

Итого у нас есть:\n
mes_ues_all - ДФ с фактическими показателями потребления по всем регионам\n
mes_ues_percent - ДФ со снижениями по ВСЕМ регионам\n
mes_UES - ДФ со снижениями по ОЭС

### UES Отклонения с 1 числа для ОПЕРАТИВНЫХ ДАННЫХ 

In [52]:
# Задаем месяц, с которого мы будем смотреть оперативные данные потребления. Оно зависит от
# количества месяцев, по которым СО дал данные о фактическом потреблении
oper_month = mes_ues_all.index[-1].month +1

In [53]:
df_uespr_otkl_oper_mon = df_uespr.loc[f'{today.year}-{oper_month}-01':]
df_ues19_otkl_oper_mon = df_ues19.loc[f'{today.year-1}-{oper_month}-01':]
df_ues19_otkl_oper_mon = df_ues19_otkl_oper_mon.iloc[:len(df_uespr_otkl_oper_mon)]

print('Длинна ряда для 20 года по температуре - ',len(df_uespr_otkl_oper_mon))
print('Длинна ряда для 19 года - ',len(df_ues19_otkl_oper_mon))

Длинна ряда для 20 года по температуре -  44
Длинна ряда для 19 года -  44


In [54]:
# Отклонение потребления с 1 июля по текущий момент

ues_delta_1_oper_date = np.round(df_uespr_otkl_oper_mon[list_UES].sum() / df_ues19_otkl_oper_mon[list_UES].sum()*100-100,1)

print(ues_delta_1_oper_date)

0
ЕЭС России          -3.1
ОЭС Центра          -1.1
ОЭС Средней Волги   -4.1
ОЭС Урала           -8.0
ОЭС Северо-Запада   -2.8
ОЭС Юга             -0.4
ОЭС Сибири          -0.2
ОЭС Востока         -0.8
dtype: float64


### UES Отклонения с 1 числа ТЕКУЩЕГО МЕСЯЦА

In [55]:
df_uespr_otkl_act_mon = df_uespr.loc[f'{today.year}-{today.month}-01':]
df_ues19_otkl_act_mon = df_ues19.loc[f'{today.year-1}-{today.month}-01':]
df_ues19_otkl_act_mon = df_ues19_otkl_act_mon.iloc[:len(df_uespr_otkl_act_mon)]

print('Длинна ряда для 20 года по температуре - ',len(df_uespr_otkl_act_mon))
print('Длинна ряда для 19 года - ',len(df_ues19_otkl_act_mon))

Длинна ряда для 20 года по температуре -  13
Длинна ряда для 19 года -  13


In [56]:
# Отклонение потребления с 1 числа текущего месяца

ues_delta_1_act_date = np.round(df_uespr_otkl_act_mon[list_UES].sum() / 
                                df_ues19_otkl_act_mon[list_UES].sum()*100-100,1)

print(ues_delta_1_act_date)

0
ЕЭС России          -2.5
ОЭС Центра          -1.6
ОЭС Средней Волги   -3.9
ОЭС Урала           -8.1
ОЭС Северо-Запада   -0.2
ОЭС Юга              3.3
ОЭС Сибири           0.0
ОЭС Востока          2.3
dtype: float64


### UES Отклонения с 1 ЯНВАРЯ

In [57]:
# Отклонение потребления с 1 янв на фактических данных

ues_delta_1jan_f = np.round((((df_uespr_otkl_oper_mon[list_UES].sum() + 
                         mes_ues_20_month['2020-01-01':][list_UES].sum()) / 
                     (df_ues19_otkl_oper_mon[list_UES].sum() + 
                      mes_ues_19_month['2019-01-01':][list_UES].sum()))*100 - 100),1)

print(ues_delta_1jan_f)

0
ЕЭС России          -2.3
ОЭС Центра          -1.3
ОЭС Средней Волги   -4.6
ОЭС Урала           -4.8
ОЭС Северо-Запада   -2.6
ОЭС Юга             -1.5
ОЭС Сибири          -0.4
ОЭС Востока          2.4
dtype: float64


### UES Отклонения с 30 МАРТА

In [58]:
# Отклонение потребления с 30 марта по по фактическим данным

ues_delta_30mar_f = np.round((((df_uespr_otkl_oper_mon[list_UES].sum() + 
                         mes_ues_20_month['2020-04-01':][list_UES].sum()) / 
                     (df_ues19_otkl_oper_mon[list_UES].sum() + 
                      mes_ues_19_month['2019-04-01':][list_UES].sum()))*100 - 100),1)

print(ues_delta_30mar_f)

0
ЕЭС России          -4.0
ОЭС Центра          -2.3
ОЭС Средней Волги   -6.5
ОЭС Урала           -7.8
ОЭС Северо-Запада   -3.7
ОЭС Юга             -2.7
ОЭС Сибири          -1.7
ОЭС Востока          0.9
dtype: float64


### UES Сведем отклонения в одну таблицу

In [59]:
ues_date_delta = pd.DataFrame()
ues_date_delta = pd.concat([ues_date_delta,ues_delta_1jan_f],axis=1)
ues_date_delta = pd.concat([ues_date_delta,ues_delta_30mar_f],axis=1)
ues_date_delta = pd.concat([ues_date_delta,ues_delta_1_act_date],axis=1)

ues_date_delta.columns = ['Отклонения с 1 января','Отклонения с 30 марта',f'Отклонения с 1.{today.month}']

### UES Сведем таблицу с месячными отклонениями потребелия в энергосистемах

Для этого соединими существующие фактические данные о помесячном потреблении и с недостающими оперативными данными

In [60]:
# Создадим отдельный датафрейм под эту благородную цель
month_ues_delta = mes_UES.round(1)

# С учетом того, за сколько месяцев сейчас есть инфа о фактическом потреблении, конкатинируем данные
if len(per_fall_month_t)-len(mes_UES) == 1:
    transit_df_2 = pd.DataFrame(ues_delta_1_act_date).T
    month_ues_delta = pd.concat([month_ues_delta,transit_df_2],ignore_index=True)
    
elif len(per_fall_month_t)-len(mes_UES) == 2:
    per_fall_month_t_ues = per_fall_month_t[list_UES]
    transit_df_1 = pd.DataFrame(per_fall_month_t_ues.iloc[len(mes_UES)]).T
    month_ues_delta = pd.concat([mes_UES,transit_df_1],ignore_index=True)
    transit_df_2 = pd.DataFrame(ues_delta_1_act_date).T
    month_ues_delta = pd.concat([month_ues_delta,transit_df_2],ignore_index=True)

# Округлим
month_ues_delta = month_ues_delta.astype(float).round(1)

In [61]:
# А теперь сделаем кульбит и пропишем индексы
list_number_month = ['январь','февраль','март','апрель','май','июнь',
                    'июль','август','сентябрь','октябрь','нобярь','декабрь']

list_new_month = list_number_month[:len(month_ues_delta)]
list_new_month[-1] = f'{list_new_month[-1]}' + f' до {lastday} числа'

month_ues_delta.index = list_new_month

### UES Анализ ОЭС с максимальной просадкой потребления в текущем месяце

In [62]:
# Найдем регионы с максимальным отклонением

region_min_1 = ues_delta_1_act_date.nsmallest(2).index[0]
region_min_2 = ues_delta_1_act_date.nsmallest(2).index[1]

print('Регион с наибольшим снижением потребления -',region_min_1)
print('Регион со вторым наибольшим снижением потребления -',region_min_2)

Регион с наибольшим снижением потребления - ОЭС Урала
Регион со вторым наибольшим снижением потребления - ОЭС Средней Волги


In [64]:
# Задаем списки регионов с минимальным отклонением
list_region_min_1 = list(region_df[region_df['UES']==region_min_1]['region'].unique())
list_region_min_2 = list(region_df[region_df['UES']==region_min_2]['region'].unique())

# Найдем субъекты РФ с наибольшим отклонением с начала года в рамках указаных ОЭС

subjects_1_delta = (mes_ues_all.loc['2020'][list_region_min_1].sum() / mes_ues_all.iloc[:len(mes_ues_all.loc['2020'])][list_region_min_1].sum()*100 - 100).round(1)
subjects_1_delta = pd.DataFrame(subjects_1_delta, columns = [f'Отклонение потребления регионов в {region_min_1}']).sort_values(by=[f'Отклонение потребления регионов в {region_min_1}'])

In [65]:
# относительные колебания по 1 минимальному региону с 1 числа текущего месяца

reg_per_delta_act_mon = pd.Series(df_uespr_otkl_act_mon[list_region_min_1].sum() / 
              df_ues19_otkl_act_mon[list_region_min_1].sum())*100 - 100

reg_per_delta_act_mon = reg_per_delta_act_mon.round(1).sort_values()

In [66]:
# относительные колебания по 1 минимальному региону с 1 числа текущего месяца

reg_abs_delta_act_mon = pd.Series(df_uespr_otkl_act_mon[list_region_min_1].sum() - 
              df_ues19_otkl_act_mon[list_region_min_1].sum())

reg_abs_delta_act_mon = reg_abs_delta_act_mon.round(1).sort_values()

# Крупные промышленные потребители (индустрии)

Далее идет аналогичный анализ потребления электроэнергии крупных промышленных потребителей

### Ind Грузим две таблицы с потреблением за 2019 и 2020 года

In [67]:
df_ind_20 = pd.read_excel('Динамика_по_крупным_потребителям.xlsx', sheet_name=1) # потребление за 2020 год
df_ind_19 = pd.read_excel('Динамика_по_крупным_потребителям.xlsx', sheet_name=2) # потребление за 2019 год

# Чистим параллельно
df_ind_20 = df_ind_20.dropna(axis=0,how='all')
df_ind_19 = df_ind_19.dropna(axis=0,how='all')

# Вытаскиваем даты
columns20 = df_ind_20.iloc[0].values[3:]
columns19 = df_ind_19.iloc[0].values[3:]

# Преобразуем в серию, чтоб было проще обработать
colser20 = pd.Series(columns20)
colser19 = pd.Series(columns19)

In [68]:
# Запилим функцию по очистке даты и переводу в формат datetime для 2020 года

def make_datetime_ind_20(x):
    if isinstance(x, str):
        x = x.replace('янв','jan 2020')
        x = x.replace('фев','feb 2020')
        x = x.replace('мар','mar 2020')
        x = x.replace('апр','apr 2020')
        x = x.replace('май','may 2020')
        x = x.replace('июн','jun 2020')
        x = x.replace('июл','jul 2020')
        x = x.replace('авг','aug 2020')
        x = x.replace('сен','sep 2020')
        x = x.replace('окт','oct 2020')
        x = x.replace('ноя','nov 2020')
        x = x.replace('дек','dec 2020')
        x = dt.datetime.strptime(x, '%d %b %Y')
    return x

# Запилим функцию по очистке даты и переводу в формат datetime для 2019 года

def make_datetime_ind_19(x):
    if isinstance(x, str):
        x = x.replace('янв','jan 2019')
        x = x.replace('фев','feb 2019')
        x = x.replace('мар','mar 2019')
        x = x.replace('апр','apr 2019')
        x = x.replace('май','may 2019')
        x = x.replace('июн','jun 2019')
        x = x.replace('июл','jul 2019')
        x = x.replace('авг','aug 2019')
        x = x.replace('сен','sep 2019')
        x = x.replace('окт','oct 2019')
        x = x.replace('ноя','nov 2019')
        x = x.replace('дек','dec 2019')
        x = dt.datetime.strptime(x, '%d %b %Y')
    return x

In [69]:
# Преобразуем все элементы серии в формат datetime
colser20 = colser20.apply(make_datetime_ind_20)
colser19 = colser19.apply(make_datetime_ind_19)

# Создадим индексы из элементов datetime
new_index_ind_20 = pd.DatetimeIndex(colser20)
new_index_ind_19 = pd.DatetimeIndex(colser19)

In [70]:
# Найдем индекс, с которого начинается РЖД
indRZD20 = df_ind_20['Unnamed: 0'][df_ind_20['Unnamed: 0']=='ОАО «РЖД» (на основе данных о потреблении ООО «Русэнергосбыт»)'].index[0]
# Найдем индекс, с которого начинается Магистральные нефтепроводы
indNeft20 = df_ind_20['Unnamed: 0'][df_ind_20['Unnamed: 0']=='Магистральные нефтепроводы (на основе данных о потреблении ООО «Русэнергоресурс» и \nООО «Транснефтьэнерго»)'].index[0]

# Найдем индекс, с которого начинается РЖД
indRZD19 = df_ind_19['Unnamed: 0'][df_ind_19['Unnamed: 0']=='ОАО «РЖД» (на основе данных о потреблении ООО «Русэнергосбыт»)'].index[0]
# Найдем индекс, с которого начинается Магистральные нефтепроводы
indNeft19 = df_ind_19['Unnamed: 0'][df_ind_19['Unnamed: 0']=='Магистральные нефтепроводы (на основе данных о потреблении ООО «Русэнергоресурс» и \nООО «Транснефтьэнерго»)'].index[0]

In [71]:
df_ind_20['Unnamed: 2'][indRZD20] = 'ОАО "РЖД"'
df_ind_20['Unnamed: 2'][indNeft20] = 'Магистральные нефтепроводы'

df_ind_19['Unnamed: 2'][indRZD19] = 'ОАО "РЖД"'
df_ind_19['Unnamed: 2'][indNeft19] = 'Магистральные нефтепроводы'


# Пропишем нужные данные для колонок ОАО РЖД
for i in range(indRZD20+1,indNeft20):
    df_ind_20['Unnamed: 2'][i] = 'ОАО "РЖД" ' + df_ind_20['Unnamed: 0'][i][4:]
    df_ind_19['Unnamed: 2'][i] = 'ОАО "РЖД" ' + df_ind_19['Unnamed: 0'][i][4:]

# Пропишем нужные данные для колонок Магистральные нефтепроводы
for i in range(indNeft20+1,len(df_ind_20)+2):
    df_ind_20['Unnamed: 2'][i] = 'Магистральные нефтепроводы ' + df_ind_20['Unnamed: 0'][i][4:]
    df_ind_19['Unnamed: 2'][i] = 'Магистральные нефтепроводы ' + df_ind_19['Unnamed: 0'][i][4:]

In [72]:
df_list = [df_ind_20, df_ind_19]

# Заполняем недостающие значения в основной колонке с названиями предприятий и чистим обе таблицы
for df in df_list:
    df['Unnamed: 2'] = df['Unnamed: 2'].fillna(df['Unnamed: 0'])

In [73]:
# Cleaning

df_ind_20 = df_ind_20.drop(['Unnamed: 0','Unnamed: 1'], axis=1)
df_ind_20 = df_ind_20.dropna()
df_ind_20.reset_index(inplace=True)
df_ind_20 = df_ind_20.drop('index', axis=1)

df_ind_19 = df_ind_19.drop(['Unnamed: 0','Unnamed: 1'], axis=1)
df_ind_19 = df_ind_19.dropna()
df_ind_19.reset_index(inplace=True)
df_ind_19 = df_ind_19.drop('index', axis=1)

# Очистим названия компаний от разных кавычек и пробелов вначале и конце строки
df_ind_20['Unnamed: 2'] = df_ind_20['Unnamed: 2'].apply(new_comp)
df_ind_19['Unnamed: 2'] = df_ind_19['Unnamed: 2'].apply(new_comp)

In [74]:
# Чекаем списки компаний
list_comp_2020 = list(df_ind_20['Unnamed: 2'])
list_comp_2019 = list(df_ind_19['Unnamed: 2'])
list(set(list_comp_2020) ^ set(list_comp_2019))

[]

Списки идентичны! Значит можно пилить транспонированные таблицы

In [75]:
df_ind_20 = df_ind_20.T
df_ind_19 = df_ind_19.T

In [76]:
# Сделаем названия компаний заголовками
new_header20 = df_ind_20.iloc[0]
df_ind_20 = df_ind_20[1:]
df_ind_20.columns = new_header20
# Сделаем новые индексы
df_ind_20.index = new_index_ind_20

In [77]:
# Сделаем названия компаний заголовками
new_header19 = df_ind_19.iloc[0]
df_ind_19 = df_ind_19[1:]
df_ind_19.columns = new_header19
# Сделаем новые индексы
df_ind_19.index = new_index_ind_19

In [78]:
### Задаем список индустрий

list_industry = ['Металлургия','Алюминиевые заводы','Машиностроение','Добывающая промышленность',
                'Предприятия химической промышленности, нефтепереработка',
                 'Предприятия деревообрабатывающей промышленности','ОАО "РЖД"','Магистральные нефтепроводы']

 ### Ind Датафреймы для обработки данных по неделям

In [79]:
df_ind_20_week = df_ind_20.resample('W').sum().round(2)
df_ind_20_week.index = df_ind_20_week.index.week

In [80]:
df_ind_19_week = df_ind_19.resample('W').sum().round(2)
df_ind_19_week.index = df_ind_19_week.index.week

### Ind Датафреймы для обработки данных по месяцам

In [81]:
df_ind_19_month = df_ind_19.resample('M').sum().round(2)
df_ind_19_month.index = df_ind_19_month.index.month

df_ind_20_month = df_ind_20.resample('M').sum().round(2)
df_ind_20_month.index = df_ind_20_month.index.month

### Введем переменную последней даты в датафрейме

### Ind Грузим фактические данные помесячного потребления для индустрий

In [82]:
def new_industry(x):
    x = x.strip()
    x = x.replace('ОАО «РЖД»','ОАО "РЖД"')
    return x

# Скачиваем таблицу
mes_ind = pd.read_excel('Помесячные_приведенные_июль.xlsx', sheet_name=1)

# Удаляем ненужные столбцы
mes_ind = mes_ind.drop(['Отрасль','ОЭС','Энергосистема'],axis=1)

# Прописываем новые индексы
mes_ind.index = mes_ind['Отрасль/Предприятие']

# Очищаем от пробелов
mes_ind['Отрасль/Предприятие'] = mes_ind['Отрасль/Предприятие'].apply(new_industry)

# Удаляем столбец который перешел в индексы
mes_ind = mes_ind.drop(['Отрасль/Предприятие'],axis=1)

# Прописываем колонки как datetimeindex
mes_ind.columns = pd.DatetimeIndex(mes_ind.columns)

# Транспонируем
mes_ind = mes_ind.T

### Ind Отклонения с 1 числа по ОПЕРАТИВНЫМ ДАННЫМ

In [83]:
df_ind_20_otkl_oper_mon = df_ind_20.loc[f'{today.year}-{oper_month}-01':]
df_ind_19_otkl_oper_mon = df_ind_19.loc[f'{today.year-1}-{oper_month}-01':]
df_ind_19_otkl_oper_mon = df_ind_19_otkl_oper_mon.iloc[:len(df_ind_20_otkl_oper_mon)]

print('Динна по 2019 - ',len(df_ind_20_otkl_oper_mon))
print('Динна по 2020 - ',len(df_ind_19_otkl_oper_mon))

Динна по 2019 -  44
Динна по 2020 -  44


In [84]:
# Отклонение индустрий с 1 числа по оперативным данным
ind_delta_oper_1 = np.round((df_ind_20_otkl_oper_mon[list_industry].sum() / df_ind_19_otkl_oper_mon[list_industry].sum()*100 - 100),1)

print(ind_delta_oper_1)

Unnamed: 2
Металлургия                                                -2.6
Алюминиевые заводы                                         -0.3
Машиностроение                                             -6.7
Добывающая промышленность                                 -12.8
Предприятия химической промышленности, нефтепереработка    -1.6
Предприятия деревообрабатывающей промышленности             0.8
ОАО "РЖД"                                                  -3.6
Магистральные нефтепроводы                                -34.6
dtype: float64


### Ind Отклонения с 1 числа месяца

In [85]:
df_ind_20_otkl_act_mon = df_ind_20.loc[f'{today.year}-{today.month}-01':]
df_ind_19_otkl_act_mon = df_ind_19.loc[f'{today.year-1}-{today.month}-01':]
df_ind_19_otkl_act_mon = df_ind_19_otkl_act_mon.iloc[:len(df_ind_20_otkl_act_mon)]

print('Динна по 2019 - ',len(df_ind_20_otkl_act_mon))
print('Динна по 2020 - ',len(df_ind_19_otkl_act_mon))

Динна по 2019 -  13
Динна по 2020 -  13


In [86]:
# Отклонение индустрий с 1 числа
ind_delta_mon_1 = np.round((df_ind_20_otkl_act_mon[list_industry].sum() / df_ind_19_otkl_act_mon[list_industry].sum()*100 - 100),1)

print(ind_delta_mon_1)

Unnamed: 2
Металлургия                                                -4.0
Алюминиевые заводы                                         -0.8
Машиностроение                                             -6.1
Добывающая промышленность                                 -12.4
Предприятия химической промышленности, нефтепереработка    -4.8
Предприятия деревообрабатывающей промышленности             1.4
ОАО "РЖД"                                                  -2.2
Магистральные нефтепроводы                                -31.8
dtype: float64


### Ind Отклонение с 1 января

In [87]:
# Отклонение индустрий с 1 января
ind_delta_jan_1 = (mes_ind.loc[f'2020-01':f'2020-{oper_month-1}'][list_industry].sum() + 
         df_ind_20_otkl_oper_mon[list_industry].sum() -
         df_ind_20.loc['2020-02-29'][list_industry]) / (mes_ind.iloc[:oper_month-1][list_industry].sum() +
         df_ind_19_otkl_oper_mon[list_industry].sum())*100 - 100

ind_delta_jan_1 = ind_delta_jan_1.apply(lambda x:round(x,1))
print(ind_delta_jan_1)

Отрасль/Предприятие
Металлургия                                                -3.4
Алюминиевые заводы                                          0.7
Машиностроение                                             -5.3
Добывающая промышленность                                  -7.4
Предприятия химической промышленности, нефтепереработка    -1.7
Предприятия деревообрабатывающей промышленности             1.5
ОАО "РЖД"                                                  -6.5
Магистральные нефтепроводы                                -13.2
dtype: float64


### Ind Отклонение с 30 марта

In [88]:
df_ind_20_otkl_30mar = df_ind_20.loc['2020-03-30':]
df_ind_19_otkl_30mar = df_ind_19.loc['2019-03-30':]
df_ind_19_otkl_30mar = df_ind_19_otkl_30mar.iloc[:len(df_ind_20_otkl_30mar)]

print('Динна ряда по 2020 - ',len(df_ind_20_otkl_30mar))
print('Динна ряда по 2019 - ',len(df_ind_19_otkl_30mar))

Динна ряда по 2020 -  168
Динна ряда по 2019 -  168


In [89]:
# Отклонение индустрий с 30 марта
ind_delta_mar_30 = np.round((df_ind_20_otkl_30mar[list_industry].sum() / df_ind_19_otkl_30mar[list_industry].sum()*100 - 100),1)

print(ind_delta_mar_30)

Unnamed: 2
Металлургия                                                -3.9
Алюминиевые заводы                                          0.6
Машиностроение                                             -6.6
Добывающая промышленность                                 -10.5
Предприятия химической промышленности, нефтепереработка    -1.6
Предприятия деревообрабатывающей промышленности             0.7
ОАО "РЖД"                                                  -6.6
Магистральные нефтепроводы                                -23.2
dtype: float64


In [90]:
# Сведем все отклонения в единую табличку
ind_date_delta = pd.DataFrame()
ind_date_delta = pd.concat([ind_date_delta,ind_delta_jan_1],axis=1)
ind_date_delta = pd.concat([ind_date_delta,ind_delta_mar_30],axis=1)
ind_date_delta = pd.concat([ind_date_delta,ind_delta_mon_1],axis=1)

ind_date_delta.columns = ['Отклонения с 1 января','Отклонения с 30 марта',f'Отклонения с 1.{today.month}']

### Ind Понедельная динамика

In [91]:
# Сделаем табличку с понедельными отклонениями по индустриям
ind_weekly_delta = pd.DataFrame()
for industry in list_industry:
    transit_df = pd.DataFrame(np.round((df_ind_20_week.iloc[-15:][industry] / 
                            df_ind_19_week.loc[df_ind_20_week.iloc[-15:].index][industry]*100-100),1))
    ind_weekly_delta = pd.concat([ind_weekly_delta,transit_df],axis=1)

### Определим регионы

In [92]:
Cent = company[company['UES'] == 'ОЭС Центра']
Volg = company[company['UES'] == 'ОЭС Средней Волги']
Ural = company[company['UES'] == 'ОЭС Урала']
SZap = company[company['UES'] == 'ОЭС Северо-Запада']
Ug = company[company['UES'] == 'ОЭС Юга']
Sib = company[company['UES'] == 'ОЭС Сибири']
Vost = company[company['UES'] == 'ОЭС Востока']

Cent.name = 'ОЭС Центра'
Volg.name = 'ОЭС Средней Волги'
Ural.name = 'ОЭС Урала'
SZap.name = 'ОЭС Северо-Запада'
Ug.name = 'ОЭС Юга'
Sib.name = 'ОЭС Сибири'
Vost.name = 'ОЭС Востока'

### Ind Зададим функции для исследования данных по федеральному округу

In [93]:
# Задаем функцию для расчета относительного снижния в текущем месяце
def del_ind_per(FO):
    dict_per = {}
    for i in FO['industry'].unique():
        var_per = (df_ind_20[list(FO[FO['industry']==i]['company'])].loc[f'2020-{today.month}':].sum().sum() /\
df_ind_19[list(FO[FO['industry']==i]['company'])].loc[f'2019-{today.month}':f'2019-{today.month}-{lastday}'].sum().sum())*100 - 100
        dict_per.setdefault(i, var_per.round(1))
    return dict_per

In [94]:
# Задаем функцию для расчета абсолютного снижния в текущем месяце
def del_ind_abs(FO):
    dict_abs = {}
    for i in FO['industry'].unique():
        var_abs = (df_ind_20[list(FO[FO['industry']==i]['company'])].loc[f'2020-{today.month}':].sum().sum() -\
df_ind_19[list(FO[FO['industry']==i]['company'])].loc[f'2019-{today.month}':f'2019-{today.month}-{lastday}'].sum().sum())
        dict_abs.setdefault(i, var_abs.round(1))
    return dict_abs

In [95]:
# Задаем функцию для расчета абсолютного снижния с 30 марта
def del_ind_abs_30march(FO):
    dict_abs = {}
    for i in FO['industry'].unique():
        var_abs = (df_ind_20[list(FO[FO['industry']==i]['company'])].loc[f'2020-04':].sum().sum() -\
df_ind_19[list(FO[FO['industry']==i]['company'])].loc[f'2019-04':f'2019-{today.month}-{lastday}'].sum().sum())
        dict_abs.setdefault(i, var_abs.round(1))
    return dict_abs

In [96]:
# Задаем функцию для расчета относительного снижния с 30 марта
def del_ind_per_30march(FO):
    dict_per = {}
    for i in FO['industry'].unique():
        var_per = (df_ind_20[list(FO[FO['industry']==i]['company'])].loc[f'2020-04':].sum().sum() /\
df_ind_19[list(FO[FO['industry']==i]['company'])].loc[f'2019-04':f'2019-{today.month}-{lastday}'].sum().sum())*100-100
        dict_per.setdefault(i, var_per.round(1))
    return dict_per

### Ind Исследуем ФО

In [97]:
# Задаем целевой регион - тот, у котрого самое сильное снижение потребления в текущем месяце

list_ind_FO = [Cent,Volg,Ural,SZap,Ug,Sib,Vost]

for reg in list_ind_FO:
    if reg.name == region_min_1:
        FO = reg

In [98]:
# Относительные снижения отраслей ФО
Delta_per = pd.DataFrame.from_dict(del_ind_per(FO),orient='index')
Delta_per.columns = [f'Относительные снижения потребления отраслей в {FO.name}']
Delta_per = Delta_per.sort_values(by=[f'Относительные снижения потребления отраслей в {FO.name}'], axis=0, ascending=True)

# Абсолютные снижения отраслей ФО
Delta_abs = pd.DataFrame.from_dict(del_ind_abs(FO),orient='index')
Delta_abs.columns = [f'Абсолютные снижения потребления отраслей в {FO.name}']
Delta_abs = Delta_abs.sort_values(by=[f'Абсолютные снижения потребления отраслей в {FO.name}'], axis=0, ascending=True)

In [99]:
Delta_abs

,Абсолютные снижения потребления отраслей в ОЭС Урала
Добывающая промышленность,-254.9
Магистральные нефтепроводы,-64.1
Металлургия,-33.0
"Предприятия химической промышленности, нефтепереработка",-27.7
"ОАО ""РЖД""",-15.3
Алюминиевые заводы,-0.9


In [100]:
# Находим и задаем переменную на самую упавшую в абсолютных числах индустрию ФО (кроме нефтепроводов и РЖД)

lowest_ind = Delta_abs.drop(['Магистральные нефтепроводы','ОАО "РЖД"'],axis=0).iloc[0].name
#.nsmallest(1,0).index[f'Абсолютные снижения потребления отраслей в {FO.name}']

print('Индустрия, в которой зафиксировано самое большое падение потребелния ээ:\n\n',lowest_ind)

Индустрия, в которой зафиксировано самое большое падение потребелния ээ:

 Добывающая промышленность


In [101]:
# Находим абсолютные снижения предприятий в самой упавшей индустрии

abs_fall_comp = pd.DataFrame(df_ind_20[list(FO[FO['industry']==lowest_ind]['company'])].loc[f'2020-{today.month}':].sum() -\
df_ind_19[list(FO[FO['industry']==lowest_ind]['company'])].loc[f'2019-{today.month}':f'2019-{today.month}-{lastday}'].sum())

abs_fall_comp = abs_fall_comp.astype(float).round(1).sort_values(by=0)

# Находим относительные снижения предприятий в самой упавшей индустрии

per_fall_comp = pd.DataFrame(df_ind_20[list(FO[FO['industry']==lowest_ind]['company'])].loc[f'2020-{today.month}':].sum() /\
df_ind_19[list(FO[FO['industry']==lowest_ind]['company'])].loc[f'2019-{today.month}':f'2019-{today.month}-{lastday}'].sum()*100 - 100)

per_fall_comp = per_fall_comp.astype(float).round(1).sort_values(by=0)

In [102]:
# Находим абсолютные снижения предприятий в самой упавшей индустрии за период с 30 марта

# pd.DataFrame(df_ind_20[list(FO[FO['industry']==lowest_ind]['company'])].loc[f'2020-04':].sum() -\
# df_ind_19[list(FO[FO['industry']==lowest_ind]['company'])].loc[f'2019-04':f'2019-{today.month}-{lastday}'].sum())

# Запись данных в эксель

In [103]:
# Создадим папку для записи
file_name = str(df_uespr.index[-1])[:10]+' анализ потребления'

if not os.path.isdir(file_name):
     os.mkdir(file_name)

In [104]:
with pd.ExcelWriter(os.path.join(file_name,'Total_consumption.xlsx')) as writer:  
    
    # полные отклонения по месяцам (факт + опер)
    month_ues_delta.to_excel(writer,sheet_name='Месячные отклонения факт и опер')

    # отклонения потребления по ЭС нарастающим итогом
    ues_date_delta.to_excel(writer,sheet_name='Отклонение нарастающим итогом')
    
    # колебания потребления ЕЭС России за последние 14 недель
    per_fall_week_t.iloc[-14:][list_UES].round(1).to_excel(writer,sheet_name='Недельные отклонения')
      
    # абсолютные колебания по 1 минимальному региону с 1 числа текущего месяца
    reg_abs_delta_act_mon.to_excel(writer,sheet_name=f'Абс сниж {region_min_1}')
    
    # относительные колебания по 1 минимальному региону с 1 числа текущего месяца
    reg_per_delta_act_mon.to_excel(writer,sheet_name=f'Отн сниж {region_min_1}')
    
    ind_date_delta.to_excel(writer,sheet_name='Нарастающие отклонения инд')
    
    ind_weekly_delta.to_excel(writer,sheet_name='Недельные отклонения инд')
    
    Delta_abs.to_excel(writer,sheet_name=f'Абс сниж отр {FO.name}')
    
    Delta_per.to_excel(writer,sheet_name=f'Отн сниж отр {FO.name}')
    
    abs_fall_comp.to_excel(writer,sheet_name= f'Абс {lowest_ind[:6]} {FO.name[4:]}')
    
    per_fall_comp.to_excel(writer,sheet_name= f'Отн {lowest_ind[:6]} {FO.name[4:]}')